In [1]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import os

vgg_img_width, vgg_img_height= 256, 256
resnet_img_width, resnet_img_height = 224, 224
train_data_dir = "../gajankumar1016/Artify/style_classification/train"
dev_data_dir = "../gajankumar1016/Artify/style_classification/dev"
nb_train_samples = 23400
nb_dev_samples = 1300
batch_size = 128
epochs = 7
num_classes = len(os.listdir(train_data_dir))


def create_vgg_model():
    model = applications.VGG19(
        weights="imagenet", 
        include_top=False, 
        input_shape = (vgg_img_width, vgg_img_height, 3)
    )

    for layer in model.layers[:19]:
        layer.trainable = False

    x = model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(num_classes, activation="softmax")(x)

    model_final = Model(input = model.input, output = predictions)

    adam = optimizers.Adam(lr=0.0001)
    model_final.compile(adam, loss = "categorical_crossentropy", metrics=['accuracy'])
    return model_final


def create_resnet_model():
    model = applications.ResNet50(
        weights="imagenet", 
        include_top=False, 
        input_shape = (resnet_img_width, resnet_img_height, 3)
    )

    for layer in model.layers[:-15]:
        layer.trainable = False

    x = model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(num_classes, activation="softmax")(x)

    model_final = Model(input = model.input, output = predictions)

    adam = optimizers.Adam(lr=0.0001)
    model_final.compile(adam, loss = "categorical_crossentropy", metrics=['accuracy'])
    return model_final
    
    
model = create_vgg_model()
# model = create_resnet_model()
model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [2]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
#     horizontal_flip = True,
#     vertical_flip = True,
#     rotation_range=90,
    fill_mode = "nearest",
#     zoom_range = 0.3,
#     width_shift_range = 0.3,
#     height_shift_range=0.3,
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (vgg_img_height, vgg_img_width),
    batch_size = batch_size,
    class_mode = "categorical",
    shuffle=True,
    seed=42
)


dev_datagen = ImageDataGenerator(
    rescale = 1./255
)

dev_generator = dev_datagen.flow_from_directory(
    dev_data_dir,
    target_size = (vgg_img_height, vgg_img_width),
    class_mode = "categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

print(train_generator.class_indices)
print(dev_generator.class_indices)

Found 23400 images belonging to 13 classes.
Found 1300 images belonging to 13 classes.
{'Romanticism': 11, 'Northern_Renaissance': 7, 'Abstract_Expressionism': 0, 'Cubism': 3, 'Naive_Art_Primitivism': 6, 'Realism': 9, 'Post_Impressionism': 8, 'Art_Nouveau_Modern': 1, 'Rococo': 10, 'Impressionism': 5, 'Symbolism': 12, 'Expressionism': 4, 'Baroque': 2}
{'Romanticism': 11, 'Northern_Renaissance': 7, 'Abstract_Expressionism': 0, 'Cubism': 3, 'Naive_Art_Primitivism': 6, 'Realism': 9, 'Post_Impressionism': 8, 'Art_Nouveau_Modern': 1, 'Rococo': 10, 'Impressionism': 5, 'Symbolism': 12, 'Expressionism': 4, 'Baroque': 2}


In [3]:
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


filepath= "VGG" + "_model_weights2.h5"
checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor="val_acc", verbose=1, mode='auto')
callbacks_list = [checkpoint]


history = model.fit_generator(
    train_generator,
#     samples_per_epoch = nb_train_samples,
    steps_per_epoch = nb_train_samples//batch_size,
    epochs = epochs,
    validation_data = dev_generator,
#     nb_val_samples = nb_dev_samples // batch_size,
    validation_steps=nb_dev_samples//batch_size,
#     shuffle=True,
    callbacks = callbacks_list,
    verbose=1
)

    
# Save whole model in one file:
model.save("my_model7.h5")

Instructions for updating:
Use tf.cast instead.
Epoch 1/7
182/182 [==============================] - 501s 3s/step - loss: 2.0092 - acc: 0.3125 - val_loss: 1.7513 - val_acc: 0.4008

Epoch 00001: val_acc improved from -inf to 0.40078, saving model to VGG_model_weights2.h5
Epoch 2/7
182/182 [==============================] - 361s 2s/step - loss: 1.5616 - acc: 0.4677 - val_loss: 1.6019 - val_acc: 0.4420

Epoch 00002: val_acc improved from 0.40078 to 0.44198, saving model to VGG_model_weights2.h5
Epoch 3/7
182/182 [==============================] - 350s 2s/step - loss: 1.2845 - acc: 0.5635 - val_loss: 1.5013 - val_acc: 0.4898

Epoch 00003: val_acc improved from 0.44198 to 0.48976, saving model to VGG_model_weights2.h5
Epoch 4/7
182/182 [==============================] - 349s 2s/step - loss: 0.9842 - acc: 0.6655 - val_loss: 1.5673 - val_acc: 0.4846

Epoch 00004: val_acc did not improve from 0.48976
Epoch 5/7
182/182 [==============================] - 350s 2s/step - loss: 0.6720 - acc: 0.7813